In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import drjit as dr
import mitsuba as mi

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

no_preview = False # Toggle to False to use the preview widget
                  # instead of rendering for scene visualization

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera,\
                      PathSolver, ITURadioMaterial, SceneObject

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('data.csv',sep = ' ')
data

In [ ]:
scene = load_scene('custom_scene.xml', merge_shapes=True) # Objects are merged by default
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="cross",
                             )

# Create transmitter

car_material = ITURadioMaterial("car-material",
                                "metal",
                                thickness=0.01,
                                color=(0.8, 0.1, 0.1))

cars = [SceneObject(fname = sionna.rt.scene.low_poly_car, name = f'car{i}',radio_material = car_material) for i in range(2)]

scene.edit(add = cars)


cars[0].position = mi.Point3f(float(data['Car1x'][0]),float(data['Car1y'][0]),float(data['Car1z'][0]))
cars[1].position = mi.Point3f(float(data['Car2x'][0]),float(data['Car2y'][0]),float(data['Car2z'][0]))


rx = Receiver(name="rx",
              position=[float(data['Car1x'][0]),float(data['Car1y'][0]),float(data['Car1z'][0])+2],
              display_radius=1)
tx = Transmitter(name="tx",
                 position=[float(data['Car2x'][0]),float(data['Car2y'][0]),float(data['Car2z'][0])+2],
                 display_radius=1)
scene.add(rx)
scene.add(tx)

p_solver  = PathSolver()
paths = p_solver(scene=scene,
                 max_depth=10,
                 los=True,
                 specular_reflection=True,
                 diffuse_reflection=False,
                 refraction=True,
                 synthetic_array=False,
                 seed=41)
if no_preview:
    scene.render(camera=cam,paths = paths);
else:
    scene.preview(paths = paths);

In [ ]:
rssi =[]
cam = Camera(position=[314,112,62], look_at=[88,52,25.7])
for i in range (1,100):
    print(i)
    
    p_solver  = PathSolver()
    paths = p_solver(scene=scene,
                 max_depth=3,
                 los=True,
                 specular_reflection=True,
                 diffuse_reflection=True,
                 refraction=True,
                 synthetic_array=False,
                 seed=41)
    a,_ = paths.cir(normalize_delays = False, out_type = 'numpy')

    try:

        scene.render_to_file(camera=cam,
                             filename=f"results/renders3/frame{i}.png",
                             resolution=[650,500],paths = paths);
        path_powers = np.abs(a)**2
        total_power = np.sum(path_powers) 
        rssi.append(total_power)
        print('here')
    except:
        scene.render_to_file(camera=cam,
                             filename=f"results/renders2/frame{i}.png",
                             resolution=[650,500]);
        rssi.append(0)
    for j in range(len(cars)):
        cars[j].position = mi.Point3f(float(data[f'Car{j+1}x'][i]),float(data[f'Car{j+1}y'][i]),float(data[f'Car{j+1}z'][i]))
    scene.get('tx').position = mi.Point3f(float(data['Car1x'][i]),float(data['Car1y'][i]),float(data['Car1z'][i])+2)

    scene.get('rx').position = mi.Point3f(float(data['Car2x'][i]),float(data['Car2y'][i]),float(data['Car2z'][i])+2)
    
print('Finish')

In [ ]:
import matplotlib.pyplot as plt
new_rssi=[]
for i in range(len(rssi)):
    new_rssi.append(10 * np.log10(rssi[i]))

def sma(arr, window_size:int=11):

    new_arr = list(arr.copy())
    for i in range(window_size//2):
        new_arr.insert(0,new_arr[0])
        new_arr.append(new_arr[-1])

    sma = np.convolve(new_arr,np.ones(window_size)/window_size,mode='valid')
    return sma
fig,ax = plt.subplots()
ax.plot(rssi)


In [ ]:
# Number of cars to add
num_cars = 10

# Radio material constituing the cars
# We use ITU metal, and use red color for visualization to
# make the cars easily discernible
car_material = ITURadioMaterial("car-material",
                                "metal",
                                thickness=0.01,
                                color=(0.8, 0.1, 0.1))

# Instantiate `num_cars` cars sharing the same mesh and material
cars = [SceneObject(fname=sionna.rt.scene.low_poly_car, # Simple mesh of a car
                    name=f"car-{i}",
                    radio_material=car_material)
        for i in range(num_cars)]

# Add the list of newly instantiated objects to the scene
scene.edit(add=cars)

if no_preview:
    scene.render(camera=cam);
else:
    scene.preview();

In [ ]:
# Positions
# Car are positioned in a circle around the central monument
# Center of the circle
c = mi.Point3f(-127, 37, 1.5)
# Radius of the circle
r = 100
# Angles at which cars are positioned
thetas = dr.linspace(mi.Float, 0., dr.two_pi, num_cars, endpoint=False)
# Cars positions
cars_positions = c + mi.Point3f(dr.cos(thetas), dr.sin(thetas), 0.)*r

# Orientations
# Compute points the car "look-at" to set their orientation
d = dr.normalize(cars_positions - c)
# Tangent vector to the circle at the car position
look_at_dirs = mi.Vector3f(d.y, -d.x, 0.)
look_at_points = cars_positions + look_at_dirs

# Set the cars positions and orientations
for i in range(num_cars):
    cars[i].position = mi.Point3f(-27, 37, 1.5)
    cars[i].look_at(mi.Point3f(look_at_points.x[i], look_at_points.y[i], look_at_points.z[i]))

if no_preview:
    scene.render(camera=cam);
else:
    scene.preview();

In [ ]:
mi.Point3f(cars_positions.x[0], cars_positions.y[0], cars_positions.z[0])